## 1 sobel理论基础 
用来计算水平方向的梯度和垂直方向的梯度，如果梯度很大，说明该处为边界，如果很小，说明不是边界

如[[p1,p2,p3],[p4,p5,p6],[p7,p8,p9]]

水平方向的梯度：右边的一列减去左边的一列，距离近一点的权重大一些，如P5x=（p3-p1）+2*(p6-p4)+(p9-p7)

垂直方向的梯度：下边的一行减去上边的一行，距离近一点的权重大一些，如P5x=(p7-p1)+2*(p8-p2)+(p9-p3)

平均梯度：G=sqrt(P5x^2+P5y^2),简化版本为G=|P5x|+|P5y|

## 2 sobel算子及其函数运用
格式：dst=cv2.Sobel(src,ddepth,dx,dy,[ksize])

dst:计算结果

src：原始图像

ddepth：处理结果图像深度，通常设置为-1，让处理结果与原始图像保持一致
注意：在实际操作时，对于水平梯度，如果右边一列与左边一列的差值不为0,是边界，为0时不是边界;对于垂直梯度，如果下边一行与上边一行的差值不为0,是边界，为0时不是边界。但是对于8位图像来说，像素值范围为0～256，如果差值为负数，将自动变为0,则会使是边界误判为不是边界。针对这种情况，
ddepth通常使用更高的数据类型cv2.CV_64F,取绝对值后，再转换为np.uint8（cv2.CV_8U)类型，使用如下函数直接调整：

目标图像=cv2.convertScaleAbs(原始图像）

dx：x轴方向的边界，计算x方向梯度：[dx=1,dy=0]

dy：y轴方向的边界,计算y方向梯度：[dx=0,dy=1]

Ksize：核大小

计算sobel结果：

方法1：取dx=1，dy=1，即 dst=cv2.Sobel(src,ddepth,1,1) 该方法不够严谨,一般用方法2

方法2：分别计算dx和dy后相加：

dx=cv2.Sobel(src,ddepth,1,0)

dy=cv2.Sobel(src,ddepth,0,1)

dst=dx*系数1+dy*系数2，乘以系数可以有效避免结果大于255而溢出，通常两个系数分别取0.5

格式：dst=cv2.addWeighted(src1,alpha,src2,beta,gamma)

功能：计算两个图像的权重和

即dst=src1*alpha+src2*beta+gamma

实例：dst=cv2.addWeighted(src1,0.5,src2,0.5,0)

用于计算水平梯度和垂直梯度的3*3sobel算子分别为：

In [2]:
Gx=[[-1,0,1],[-2,0,2],[-1,0,1]]
Gy=[[-1,-2,-1],[0,0,0],[1,2,1]]
print(Gx)
print(Gy)

[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]


实例

In [1]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/sobel4.bmp",cv2.IMREAD_GRAYSCALE)
sobelx=cv2.Sobel(img,-1,1,0) #ddepth设为-1，即保留uint8数据类型，会导致负值被转换为0,从而时部分边界信息丢失
cv2.imshow("img",img)
cv2.imshow("sobelx",sobelx)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/sobel4.bmp",cv2.IMREAD_GRAYSCALE)
sobelx=cv2.Sobel(img,cv2.CV_64F,1,0) #ddept变换为cv2.CV_64F数据类型，防止边界信息丢失
sobelx=cv2.convertScaleAbs(sobelx) #转回uint8,此行必须要有，取绝对值后转回uint8,否则负值还是会自动变为0
cv2.imshow("img",img)
cv2.imshow("sobelx",sobelx)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/sobel4.bmp",cv2.IMREAD_GRAYSCALE)
sobely=cv2.Sobel(img,cv2.CV_64F,0,1) #求y方向的梯度
sobely=cv2.convertScaleAbs(sobely) 
cv2.imshow("img",img)
cv2.imshow("sobelx",sobely)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/sobel4.bmp",cv2.IMREAD_GRAYSCALE)
sobelx=cv2.Sobel(img,cv2.CV_64F,1,0) #求x方向的梯度
sobelx=cv2.convertScaleAbs(sobelx) 
sobely=cv2.Sobel(img,cv2.CV_64F,0,1) #求y方向的梯度
sobely=cv2.convertScaleAbs(sobely) 
sobelxy=cv2.addWeighted(sobelx,0.5,sobely,0.5,0) #将x和y方向的梯度加权求和
sobelxy1=cv2.Sobel(img,cv2.CV_64F,1,1) #直接算出，会使部分信息丢失
sobelxy1=cv2.convertScaleAbs(sobelxy1)
cv2.imshow("img",img)
cv2.imshow("sobelxy",sobelxy)
cv2.imshow("sobelxy1",sobelxy1)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 3 scharr算子
scharr算子是对sobel算子的改进，更精准，效果更好,更常用

距离近的像素权重更大

格式：dst=Scharr(src,ddepth,dx,dy)

ddept:处理方法与sobel相同，使用

dst=Scharr(src,cv2.CV_64F,dx,dy) #先转换为cv2.CV_64F格式,从而能够取得负值

dst=cv2.convertScaleAbs(dst) #再取绝对值后转换为uint8格式

dx和dy满足条件：dx>=0 && dy>=0 && dx+dy==1,所以当取dx=1,dy=1会报错

x方向梯度：dst=Scharr(src,ddepth,dx=1,dy=0)

y方向梯度：dst=Scharr(src,ddepth,dx=0,dy=1)

如果想求某一个方向的梯度，需要先分别求出x和y方向的梯度，然后再将两者加权，可用函数如：

scharrxy=cv2.addWeighted(scharrx,0.5,scharry,0.5,0)

dst=Sobel(src,ddepth,dx,dy,-1)等价于dst=Scharr(src,ddepth,dx,dy),Sobel函数中最后一个参数ksize为核的个数，可以为1、3、5等，但是如果取-1，就等价于scharr算子

In [4]:
Gx=[[-3,0,3],[-10,0,10],[-3,0,3]]
Gy=[[-3,-10,-3],[0,0,0],[3,10,3]]
print(Gx)
print(Gy)

[[-3, 0, 3], [-10, 0, 10], [-3, 0, 3]]
[[-3, -10, -3], [0, 0, 0], [3, 10, 3]]


In [1]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/scharr.bmp",cv2.IMREAD_GRAYSCALE)
scharrx=cv2.Scharr(img,cv2.CV_64F,1,0) #x方向梯度
scharrx=cv2.convertScaleAbs(scharrx)
cv2.imshow("original",img)
cv2.imshow("x",scharrx)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/scharr.bmp",cv2.IMREAD_GRAYSCALE)
scharry=cv2.Scharr(img,cv2.CV_64F,0,1) #y方向梯度
scharry=cv2.convertScaleAbs(scharry)
cv2.imshow("original",img)
cv2.imshow("y",scharry)d
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/scharr.bmp",cv2.IMREAD_GRAYSCALE)
scharrx=cv2.Scharr(img,cv2.CV_64F,1,0) #x方向梯度
scharrx=cv2.convertScaleAbs(scharrx)

scharry=cv2.Scharr(img,cv2.CV_64F,0,1) #y方向梯度
scharry=cv2.convertScaleAbs(scharry)

scharrxy=cv2.addWeighted(scharrx,0.5,scharry,0.5,0)

cv2.imshow("original",img)
cv2.imshow("x",scharrx)
cv2.imshow("y",scharry)
cv2.imshow("xy",scharrxy)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 4 sobel算子和scharr算子的比较
1）卷积核大小一样，计算量一样

2）sobel算子精确度没有scharr算子高，因为scharr算子邻近的像素权重更大

In [4]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/boat.bmp",cv2.IMREAD_GRAYSCALE)

sobelx=cv2.Sobel(img,cv2.CV_64F,1,0,ksize=3)
sobely=cv2.Sobel(img,cv2.CV_64F,0,1,ksize=3)
sobelx=cv2.convertScaleAbs(sobelx)
sobely=cv2.convertScaleAbs(sobely)
sobelxy=cv2.addWeighted(sobelx,0.5,sobely,0.5,0)

scharrx=cv2.Scharr(img,cv2.CV_64F,1,0)
scharry=cv2.Scharr(img,cv2.CV_64F,0,1)
scharrx=cv2.convertScaleAbs(scharrx)
scharry=cv2.convertScaleAbs(scharry)
scharrxy=cv2.addWeighted(scharrx,0.5,scharry,0.5,0)

cv2.imshow("original",img)
cv2.imshow("sobelxy",sobelxy)
cv2.imshow("scharrxy",scharrxy)
cv2.waitKey(0)
cv2.destroyAllWindows()


## 5 laplacian函数及其使用

拉普拉斯算子类似于二阶sobel导数，对于每个像素，分别将四个与其相邻的像素与其相减，再将结果相加，使用的卷积核为：

G=[[0,1,0],[1,-4,1],[0,1,0]]

相当于对于矩阵A=[[p1,p2,p3],[p4,p5,p6],[p7,p8,p9]],p5new=(p2-p5)+(p4-p5)+(p6-p5)+(p8-p5)=（p2+p4+p6+p8)-4*p5

如果结果很小，说明与周围像素相差很小，不是边界，如果结果很大，说明与周围点相差很大，是边界。

一阶导数：sobel算子=|左-右|+|下-上| scharr算子=|左-右|+|下-上|

二阶导数：laplacian算子=|左-右|+|左-右|+|下-上|+|下-上|

格式：dst=cv2.Laplacian(src,ddepth)

ddepth:处理方法与前面相同

In [1]:
import cv2
import numpy as np
img=cv2.imread("/home/lyc/Desktop/python+opencv/chapter10/laplacian.bmp",cv2.IMREAD_GRAYSCALE)
laplacian=cv2.Laplacian(img,cv2.CV_64F)
laplacian=cv2.convertScaleAbs(laplacian)
cv2.imshow("img",img)
cv2.imshow("laplacian",laplacian)
cv2.waitKey(0)
cv2.destroyAllWindows()